In [1]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy
import os
import matplotlib as mpl
import matplotlib.tri as tri
import ternary
import pickle
import datetime
from collections import Counter
import matplotlib.ticker as ticker
from sklearn import preprocessing
import pyDOE
import random
from scipy.stats import norm
import time
from sklearn.ensemble import RandomForestRegressor
import copy

In [2]:
AutoAM_dataset = pd.read_csv("~/Downloads/AutoAM_dataset.csv", encoding = 'utf-8')

In [3]:
dataset_name = 'AutoAM'
AutoAM_feature_name = list(AutoAM_dataset.columns)[:-1]
AutoAM_objective_name = list(AutoAM_dataset.columns)[-1]

In [4]:
AutoAM_ds = copy.deepcopy(AutoAM_dataset) 
# only P3HT/CNT, Crossed barrel, AutoAM need this line; Perovskite and AgNP do not need this line.
AutoAM_ds[AutoAM_objective_name] = -AutoAM_dataset[AutoAM_objective_name].values

In [5]:
AutoAM_ds_grouped = AutoAM_ds.groupby(AutoAM_feature_name)[AutoAM_objective_name].agg(lambda x: x.unique().mean())
AutoAM_ds_grouped = (AutoAM_ds_grouped.to_frame()).reset_index()
AutoAM_ds_grouped

,Prime Delay,Print Speed,X Offset Correction,Y Offset Correction,Score
0,0.0,0.1,-0.837284,-1.000000,-0.138480
1,0.0,0.1,-0.562308,-1.000000,-0.553860
2,0.0,0.1,-0.399772,1.000000,-0.237950
3,0.0,0.1,-0.383307,-0.025953,-0.529015
4,0.0,0.1,-0.263441,-1.000000,-0.444846
...,...,...,...,...,...
95,5.0,10.0,-0.453751,-1.000000,-0.649364
96,5.0,10.0,-0.392564,-1.000000,-0.641716
97,5.0,10.0,-0.030374,-1.000000,-0.394429
98,5.0,10.0,1.000000,-1.000000,0.000000


In [6]:
# these are the input feature x and objective value y used in framework
AutoAM_X_feature = AutoAM_ds_grouped[AutoAM_feature_name].values
AutoAM_y = np.array(AutoAM_ds_grouped[AutoAM_objective_name].values)
assert len(AutoAM_ds_grouped) == len(AutoAM_X_feature) == len(AutoAM_y)
# total number of data in set
AutoAM_N = len(AutoAM_ds_grouped)
print(AutoAM_N)

100


In [7]:
# here are some parameters of the framework, feel free to modify for your own purposes
# number of ensembles. in the paper n_ensemble = 50.
AutoAM_n_ensemble = 50
# number of initial experiments
AutoAM_n_initial = 2
# number of top candidates, currently using top 5% of total dataset size
AutoAM_n_top = int(math.ceil(len(AutoAM_y) * 0.05))
# the top candidates and their indicies
AutoAM_top_indices = list(AutoAM_ds_grouped.sort_values(AutoAM_objective_name).head(AutoAM_n_top).index)
# random seeds used to distinguish between different ensembles
# there are 300 of them, but only first n_ensemble are used
seed_list = [4295, 8508, 326, 3135, 1549, 2528, 1274, 6545, 5971, 6269, 2422, 4287, 9320, 4932, 951, 4304, 1745, 5956, 7620, 4545, 6003, 9885, 5548, 9477, 30, 8992, 7559, 5034, 9071, 6437, 3389, 9816, 8617, 3712, 3626, 1660, 3309, 2427, 9872, 938, 5156, 7409, 7672, 3411, 3559, 9966, 7331, 8273, 8484, 5127, 2260, 6054, 5205, 311, 6056, 9456, 928, 6424, 7438, 8701, 8634, 4002, 6634, 8102, 8503, 1540, 9254, 7972, 7737, 3410, 4052, 8640, 9659, 8093, 7076, 7268, 2046, 7492, 3103, 3034, 7874, 5438, 4297, 291, 5436, 9021, 3711, 7837, 9188, 2036, 8013, 6188, 3734, 187, 1438, 1061, 674, 777, 7231, 7096, 3360, 4278, 5817, 5514, 3442, 6805, 6750, 8548, 9751, 3526, 9969, 8979, 1526, 1551, 2058, 6325, 1237, 5917, 5821, 9946, 5049, 654, 7750, 5149, 3545, 9165, 2837, 5621, 6501, 595, 3181, 1747, 4405, 4480, 4282, 9262, 6219, 3960, 4999, 1495, 6007, 9642, 3902, 3133, 1085, 3278, 1104, 5939, 7153, 971, 8733, 3785, 9056, 2020, 7249, 5021, 3384, 8740, 4593, 7869, 9941, 8813, 3688, 8139, 6436, 3742, 5503, 1587, 4766, 9846, 9117, 7001, 4853, 9346, 4927, 8480, 5298, 4753, 1151, 9768, 5405, 6196, 5721, 3419, 8090, 8166, 7834, 1480, 1150, 9002, 1134, 2237, 3995, 2029, 5336, 7050, 6857, 8794, 1754, 1184, 3558, 658, 6804, 8750, 5088, 1136, 626, 8462, 5203, 3196, 979, 7419, 1162, 5451, 6492, 1562, 8145, 8937, 8764, 4174, 7639, 8902, 7003, 765, 1554, 6135, 1689, 9530, 1398, 2273, 7925, 5948, 1036, 868, 4617, 1203, 7680, 7, 93, 3128, 5694, 6979, 7136, 8084, 5770, 9301, 1599, 737, 7018, 3774, 9843, 2296, 2287, 9875, 2349, 2469, 8941, 4973, 3798, 54, 2938, 4665, 3942, 3951, 9400, 3094, 2248, 3376, 1926, 5180, 1773, 3681, 1808, 350, 6669, 826, 539, 5313, 6193, 5752, 9370, 2782, 8399, 4881, 3166, 4906, 5829, 4827, 29, 6899, 9012, 6986, 4175, 1035, 8320, 7802, 3777, 6340, 7798, 7705]

In [8]:
n_est = 100

In [9]:
def RF_pred(X, RF_model):
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    return mean, std

def EI(X, RF_model, y_best):
    mean, std = RF_pred(X, RF_model)    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model, ratio):    
    mean, std = RF_pred(X, RF_model)
    return - mean + ratio * std

def PI(X, RF_model, y_best):    
    mean, std = RF_pred(X, RF_model)    
    z = (y_best - mean)/std
    return norm.cdf(z)

In [10]:
# framework
# good practice to keep check of time used
start_time = time.time()
# these will carry results along optimization sequence from all n_ensemble runs
index_collection = []
X_collection = []
y_collection = []
TopCount_collection = []
for s in seed_list:
    if len(index_collection) == AutoAM_n_ensemble:
        break
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(AutoAM_N))
# index_learn is the pool of candidates to be examined
    index_learn = indices.copy()
# index_ is the list of candidates we have already observed
#     adding in the initial experiments
    index_ = random.sample(index_learn, AutoAM_n_initial)
#     list to store all observed good candidates' input feature X
    X_ = []
#     list to store all observed good candidates' objective value y
    y_ = []
#     number of top candidates found so far
    c = 0
#     list of cumulative number of top candidates found at each learning cycle
    TopCount_ = []
#     add the first n_initial experiments to collection
    for i in index_:
        X_.append(AutoAM_X_feature[i])
        y_.append(AutoAM_y[i])
        if i in AutoAM_top_indices:
            c += 1
        TopCount_.append(c)
        index_learn.remove(i)
#     for each of the the rest of (N - n_initial) learning cycles
#     this for loop ends when all candidates in pool are observed 
    for i in np.arange(len(index_learn)):
        y_best = np.min(y_)
        s_scaler = preprocessing.StandardScaler()
        X_train = s_scaler.fit_transform(X_)
        y_train = s_scaler.fit_transform([[i] for i in y_])
        RF_model = RandomForestRegressor(n_estimators= n_est, n_jobs= -1)
        RF_model.fit(X_train, y_train)
#         by evaluating acquisition function values at candidates remaining in pool
#         we choose candidate with larger acquisition function value to be observed next   
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = AutoAM_X_feature[j]
            y_j = AutoAM_y[j]
#             #TODO: select Acquisiton Function for BO
            ac_value = LCB(X_j, RF_model, 10)
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
        X_.append(AutoAM_X_feature[next_index])
        y_.append(AutoAM_y[next_index])
        if next_index in AutoAM_top_indices:
            c += 1
        TopCount_.append(c)
        index_learn.remove(next_index)
        index_.append(next_index)        
    assert len(index_) == AutoAM_N
    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopCount_collection.append(TopCount_)
    print('Finished seed')
total_time = time.time() - start_time
AutoAM_master = np.array([index_collection, X_collection, y_collection, TopCount_collection, total_time])
#  #TODO: name output file
np.save('AutoAM_RF_test_run', AutoAM_master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin